In [ ]:
import sys
sys.path.append('..\install\lib')
import mnpy as mn
import reader
import metnum as metnum

import numpy as np
from matplotlib import pyplot
import sklearn.model_selection as sklms


In [ ]:
df = reader.read7Zip('../data/train.7z')
targets = df['label'].to_numpy()
# la primer columna es el label, el resto son los 28*28 pixeles de cada imagen
images = df.iloc[:, 1:].to_numpy()

cols = 10
rows = 4
samples = rows*cols
titles = ["Dígito %d" % digit for digit in targets[0:samples] ]
metnum.plot_gallery(images[0:samples], titles, 28, 28, rows, cols)
pyplot.show()

In [ ]:
pca = mn.PCA(64, 120)
pca.tolerance_error = 0.000001
training_dataset = images[0:4000]
training_targets = targets[0:4000]

pca.fit(training_dataset)

knn = mn.kNNClassifier(4, mn.KNNWeights.Uniform)
knn.fit(pca.transform(training_dataset), training_targets)

predict_dataset = images[5000:5040]
predicted_tagets = knn.predict(pca.transform(predict_dataset))

titles = ["Digito %d" % digit for digit in predicted_tagets[0:samples] ]
metnum.plot_gallery(predict_dataset, titles, 28, 28, rows, cols)
pyplot.show()


In [ ]:

# df = reader.read7Zip('../data/train.7z')
# all_targets = df['label'].to_numpy()
# # la primer columna es el label, el resto son los 28*28 pixeles de cada imagen
# all_images = df.iloc[:, 1:].to_numpy()

# targets = all_targets[0:5000]
# images = all_images[0:5000]

# kfold = sklms.KFold(n_splits=10, shuffle=True)
# n_components = [8, 12, 16, 32]
# iterated_powers = [128]
# tolerance_errors = [0.0001]
# k_neighbors = [4]
# search = metnum.GridSearchCV(n_components, iterated_powers, tolerance_errors, k_neighbors, cv=kfold)
# search.fit(images, targets)
# print("Mejores parametros: ", search.best_params)
# print("Todos los resultados: ", search.results)



Momento de graficar.

In [ ]:
df = reader.read7Zip('../data/train.7z')
all_targets = df['label'].to_numpy()
# la primer columna es el label, el resto son los 28*28 pixeles de cada imagen
all_images = df.iloc[:, 1:].to_numpy()

targets = all_targets[0:5000]
images = all_images[0:5000]

kfold = sklms.KFold(n_splits=10, shuffle=True)
knn_params = [{ 
        'k_neighbor' : [1, 2, 3, 4, 5],
        # 'weights' : [0, 1, 2],
    }]

pca_params = [{ 
        # 'n_component' : [8, 16, 24, 32, ],
        # 'iterated_power' : [64, 96],
        'n_component' : [8, 16, 32, 48, 64 ],
        'iterated_power' : [32, 64, 96],
    }]

search = metnum.NewGridSearchCV(pca_params, knn_params, cv=kfold)
search.fit(images, targets)
# print("Mejores parametros: ", search.best_params)
# print("Todos los resultados: ", search.results)


In [ ]:
best_result = {}
scores = {}
for result in search.results:
    n_component = result['pca_parameters']['n_component']
    iterated_power = result['pca_parameters']['iterated_power']
    k_neighbor = result['knn_parameters']['k_neighbor']
    key = (n_component, iterated_power, k_neighbor)
    if not key in scores:
        scores[key] = []
    
    scores[key].append(result['score'])

for [key, score] in scores.items():
    mean = sum(score) / len(score)
    print(key, mean)
